In [4]:
import pandas as pd 
from sklearn.cluster import KMeans
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# %matplotlib notebook

from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

%pylab inline
import math as mt
import csv
from sparsesvd import sparsesvd #used for matrix factorization
from scipy.sparse import csc_matrix #used for sparse matrix
from scipy.sparse.linalg import * #used for matrix multiplication

/home/user/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


/home/user/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['sqrt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [5]:
dataset1 = '/home/user/257/10000.txt'
dataset2 = '/home/user/257/song_data.csv'

temp_df1 = pd.read_csv(dataset1, sep="\t", names =["user_id", "song_id", "listen_count"])
temp_df2 = pd.read_csv(dataset2)
#temp_df1.head()
#temp_df2.head()


final_df = pd.merge(temp_df1, temp_df2.drop_duplicates(['song_id']), on="song_id", how="left")
final_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [6]:
final_df["user_id"] = pd.Categorical(final_df["user_id"])
final_df['user_id'] = final_df["user_id"].cat.codes

final_df["song_id"] = pd.Categorical(final_df["song_id"])
final_df['song_id'] = final_df["song_id"].cat.codes

final_df.sort_values(['user_id'], ascending=[1])

final_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,54961,153,1,The Cove,Thicker Than Water,Jack Johnson,0
1,54961,413,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,54961,736,1,Stronger,Graduation,Kanye West,2007
3,54961,750,1,Constellations,In Between Dreams,Jack Johnson,2005
4,54961,1188,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [7]:
df = final_df[['user_id', 'song_id', 'listen_count']]

In [8]:
df.head()

,user_id,song_id,listen_count
0,54961,153,1
1,54961,413,2
2,54961,736,1
3,54961,750,1
4,54961,1188,1


In [9]:
df['Ratings'] = np.random.randint(1, 6, df.shape[0])

/home/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df.head()

,user_id,song_id,listen_count,Ratings
0,54961,153,1,1
1,54961,413,2,3
2,54961,736,1,5
3,54961,750,1,2
4,54961,1188,1,5


In [11]:
song_df = final_df[['song_id', 'title', 'artist_name']]
song_df.head()

,song_id,title,artist_name
0,153,The Cove,Jack Johnson
1,413,Entre Dos Aguas,Paco De Lucia
2,736,Stronger,Kanye West
3,750,Constellations,Jack Johnson
4,1188,Learn To Fly,Foo Fighters


In [12]:
ratings_df = df[['user_id', 'song_id', 'Ratings']]

In [13]:
ratings_df.head()

,user_id,song_id,Ratings
0,54961,153,1
1,54961,413,3
2,54961,736,5
3,54961,750,2
4,54961,1188,5


In [14]:
R_df = ratings_df.pivot(index = 'user_id', columns ='song_id', values = 'Ratings').fillna(0)
R_df.head()

song_id,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [16]:
U, s, Vt = svds(R_demeaned, k = 50)

In [17]:
s = np.diag(s)

In [18]:
all_user_predicted_ratings = np.dot(np.dot(U, s), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [97]:
def recommend_songs(predictions_df, user_id, song_df, original_ratings_df, num_recommendations=10):
    
    # Get and sort the user's predictions
    user_row_number = user_id - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    
    # Get the user's data and merge in the song information.
    user_data = original_ratings_df[original_ratings_df.user_id == (user_id)]
    user_full = (user_data.merge(song_df, how = 'left', left_on = 'song_id', right_on = 'song_id').
                     sort_values(['Ratings'], ascending=False)
                 )
    
    user_full = user_full.drop_duplicates()
    

    print("User {0} has already rated {1} songs.".format(user_id, user_full.shape[0]))
    print("Recommending the highest {0} predicted ratings songs not already rated.".format(num_recommendations))
    
    # Recommend the highest predicted rating songs that the user hasn't seen yet.
    recommendations = (song_df[~song_df['song_id'].isin(user_full['song_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'song_id',
               right_on = 'song_id').
         rename(columns = {user_row_number: 'Predictions'}).drop_duplicates().
         sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1])


    return user_full, recommendations

already_rated, predictions = recommend_songs(preds_df, 837, song_df, ratings_df, 10)

User 837 has already rated 50 songs.
Recommending the highest 10 predicted ratings songs not already rated.


In [98]:
already_rated.head(10)

,user_id,song_id,Ratings,title,artist_name
57504,837,6230,5,Billionaire [feat. Bruno Mars] (Explicit Albu...,Travie McCoy
52754,837,5888,5,Spell,Nick Cave & The Bad Seeds
92903,837,8944,5,Verde Que Te Quero Rosa,Cartola
16356,837,1354,5,Breakeven,The Script
23639,837,1686,5,L.E.D. There Be Light,Rank 1
55587,837,6175,4,Pursuit Of Happiness (nightmare),Kid Cudi / MGMT / Ratatat
22969,837,1664,4,Horn Concerto No. 4 in E flat K495: II. Romanc...,Barry Tuckwell/Academy of St Martin-in-the-Fie...
73296,837,7416,4,Revelry,Kings Of Leon
42129,837,3074,4,Catch You Baby (Steve Pitron & Max Sanna Radio...,Lonnie Gordon
83360,837,8092,4,OMG,Usher featuring will.i.am


In [99]:
predictions

,song_id,title,artist_name
1277,9096,Unite (2009 Digital Remaster),Beastie Boys
2435,1994,Nothing Better (Album),Postal Service
3093,5399,Strut (1993 Digital Remaster),Sheena Easton
82,8610,16 Candles,The Crests
59,1225,Jamaica Roots II(Agora E Sempre),Natiruts
649,8019,Gears,Future Rock
5201,4806,Who Can Compare,Foolish Things
2918,2852,West One (Shine On Me),The Ruts
2718,5928,I Think I'll Live,Charlie Louvin
3226,9708,A Beggar On A Beach Of Gold,Mike And The Mechanics
